In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, log_loss

In [2]:
param_array = pd.read_csv("parameters/parameter_array.csv").set_index("PARAM")

In [3]:
Train_Loss = []
Val_Loss = []
Test_Loss = []

Train_Acc = []
Val_Acc = []
Test_Acc = []

Train_F1 = []
Val_F1 = []
Test_F1 = []
for param in param_array.index:
    
    try:
        y_train = np.genfromtxt(f"Results/Result_{param}/y_train_truth.csv", delimiter=',')
        y_val = np.genfromtxt(f"Results/Result_{param}/y_val_truth.csv", delimiter=',')
        y_test = np.genfromtxt(f"Results/Result_{param}/y_test_truth.csv", delimiter=',')

        train_pred = np.genfromtxt(f"Results/Result_{param}/y_train_pred.csv", delimiter=',')
        val_pred = np.genfromtxt(f"Results/Result_{param}/y_val_pred.csv", delimiter=',')
        test_pred = np.genfromtxt(f"Results/Result_{param}/y_test_pred.csv", delimiter=',')

        train_loss = log_loss(y_train.flatten(), train_pred.flatten())
        val_loss = log_loss(y_val.flatten(), val_pred.flatten())
        test_loss = log_loss(y_test.flatten(), test_pred.flatten())

        train_accuracy = accuracy_score(np.argmax(y_train,axis=1), np.argmax(train_pred,axis=1))
        val_accuracy = accuracy_score(np.argmax(y_val,axis=1), np.argmax(val_pred,axis=1))
        test_accuracy = accuracy_score(np.argmax(y_test,axis=1), np.argmax(test_pred,axis=1))

        train_f1 = f1_score(np.argmax(y_train,axis=1), np.argmax(train_pred,axis=1), average='weighted')
        val_f1 = f1_score(np.argmax(y_val,axis=1), np.argmax(val_pred,axis=1), average='weighted')
        test_f1 = f1_score(np.argmax(y_test,axis=1), np.argmax(test_pred,axis=1), average='weighted')
        
    except:
        
        train_loss = np.nan
        val_loss = np.nan
        test_loss = np.nan

        train_accuracy = np.nan
        val_accuracy = np.nan
        test_accuracy = np.nan

        train_f1 = np.nan
        val_f1 = np.nan
        test_f1 = np.nan
        
    Train_Loss.append(train_loss)
    Val_Loss.append(val_loss)
    Test_Loss.append(test_loss)

    Train_Acc.append(train_accuracy)
    Val_Acc.append(val_accuracy)
    Test_Acc.append(test_accuracy)

    Train_F1.append(train_f1)
    Val_F1.append(val_f1)
    Test_F1.append(test_f1)
    
param_array["Train_Loss"] = Train_Loss
param_array["Val_Loss"] = Val_Loss
param_array["Test_Loss"] = Test_Loss

param_array["Train_Acc"] = Train_Acc
param_array["Val_Acc"] = Val_Acc
param_array["Test_Acc"] = Test_Acc

param_array["Train_F1"] = Train_F1
param_array["Val_F1"] = Val_F1
param_array["Test_F1"] = Test_F1

In [4]:
param_array.groupby(["CONFIG_SUPPORT", "DATA"]).mean()["Test_Loss"]

CONFIG_SUPPORT  DATA 
0               PD       0.049044
                PD+SM         NaN
                SM            NaN
1               PD            NaN
                PD+SM         NaN
                SM            NaN
Name: Test_Loss, dtype: float64